## Setup

Before running this norebook make sure you have updated the `.env` file for your Airtable API token and the ID for the Airtable Base you are using. Those parameters will be read in as a part of the `airtable.py` file to authenticate the API requests.

You will also need to have an Airtable base with a sheet named `golf-scores` containing the following columns and data types:

- Date (string)
- Hole (integer)
- Par (integer)
- Score (integer)

Once that's done let's get into the code by first importing the requirements and loading the initial data we'll use.

For this demo we'll use the file `data/initial_data.json` that contains a set of scores for a round of golf.

In [56]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv()

AIRTABLE_TOKEN = os.getenv("AIRTABLE_TOKEN")
AIRTABLE_BASE_ID = os.getenv("AIRTABLE_BASE_ID")
AIRTABLE_URL = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}"

# Load initial data.
data = json.load(open("data/initial_data.json", "r"))

print(data)

{'records': [{'fields': {'Date': '2021-10-08', 'Hole': 1, 'Par': 4, 'Score': 5}}, {'fields': {'Date': '2021-10-08', 'Hole': 2, 'Par': 4, 'Score': 3}}, {'fields': {'Date': '2021-10-08', 'Hole': 3, 'Par': 5, 'Score': 6}}, {'fields': {'Date': '2021-10-08', 'Hole': 4, 'Par': 3, 'Score': 2}}, {'fields': {'Date': '2021-10-08', 'Hole': 5, 'Par': 4, 'Score': 4}}, {'fields': {'Date': '2021-10-08', 'Hole': 6, 'Par': 4, 'Score': 6}}, {'fields': {'Date': '2021-10-08', 'Hole': 7, 'Par': 4, 'Score': 4}}, {'fields': {'Date': '2021-10-08', 'Hole': 8, 'Par': 5, 'Score': 4}}, {'fields': {'Date': '2021-10-08', 'Hole': 9, 'Par': 4, 'Score': 3}}, {'fields': {'Date': '2021-10-08', 'Hole': 10, 'Par': 4, 'Score': 5}}, {'fields': {'Date': '2021-10-08', 'Hole': 11, 'Par': 4, 'Score': 3}}, {'fields': {'Date': '2021-10-08', 'Hole': 12, 'Par': 5, 'Score': 6}}, {'fields': {'Date': '2021-10-08', 'Hole': 13, 'Par': 3, 'Score': 2}}, {'fields': {'Date': '2021-10-08', 'Hole': 14, 'Par': 4, 'Score': 4}}, {'fields': {'Dat

## Adding Records to Airtable

The `add_new_scores` function accepts an object, `scores`, that will be added to the Airtable. The variables read from the `.env` file are used to authenticate the `POST` request.

In [57]:
def add_new_scores(scores):
    """Add scores to the Airtable."""
    # Check if more than 10 records are trying to be added.
    if len(scores["records"]) > 10:
        print("Cannot add more than 10 records in a single request.")
        return None

    url = f"{AIRTABLE_URL}/golf-scores"
    headers = {
      'Authorization': f'Bearer {AIRTABLE_TOKEN}',
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=json.dumps(scores))

    return response

The first request using `add_new_scores` will add a single record. The `small_data` variable has data for a single fictional hole number 0 that will be passed to `add_new_scores`. After running the cell look at your Airtable to confirm the record is there.

In [58]:
# Add one record to Airtable.
small_data = {
    "records": [
        {
            "Date": "2021-10-20",
            "Hole": 0,
            "Par": 4,
            "Score": 6
        }
    ]
}

response = add_new_scores(small_data)

One limitation to Airtable is you can only submit up to 10 records at a time. When you try adding more than 10 records you'll get an error. Let's try adding all 18 records in `data` to the table and see what happens.

In [59]:
# Adding multiple records (this will be an error!). 
response = add_new_scores(data)

Cannot add more than 10 records in a single request.


If you want to add more than 10 records you will have to add the records by submitting multiple requests. Below the `chunk` function breaks the `data` object into segments of 10 records and will iteratively call the `add_new_scores` function to add each chunk to Airtable.

In [60]:
# Chunk the data into groups of no more than 10. Submit each chunk separately.
def chunk(arr, n):
    for i in range(0, len(arr), n):
        yield arr[i:i + n]


for c in chunk(data["records"], 10):
    chunk_data = {
        "records": c
    }

    response = add_new_scores(chunk_data)
    print(response)

<Response [200]>
<Response [200]>


## Reading Records from Airtable

Now that there are records in the Airtable let's look at how to read them. The `get_golf_scores` function will retrieve records from Airtable.

In [61]:
def get_golf_scores():
    """Retrieve records from Airtable."""
    url = f"{AIRTABLE_URL}/golf-scores"
    headers = {
      'Authorization': f'Bearer {AIRTABLE_TOKEN}',
      'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers)

    return response

In [62]:
# Read records from Airtable.
output = get_golf_scores()

print(output)

<Response [200]>


The result of `output` will list all the records in Airtable. One limitation of Airtable is only a maximum of 100 records are returned with each request. If you want to retrieve more than 100 records you will need to leverage the `offset` parameter returned by Airtable and issue multiple requests.

The function `get_golf_scores_by_page` accepts an optional parameter `offset`. If provided, the `offset` value will be added to the request and Airtable will return the next page of results. Unlike `get_golf_scores` we're also including a `params` object in the request that dictates each request should return 5 records so that the pagination can be tested.

In [63]:
def get_golf_scores_by_page(offset=None):
    """Retrieve records from Airtable and apply offset is necessary."""
    url = f"{AIRTABLE_URL}/golf-scores"
    headers = {
      'Authorization': f'Bearer {AIRTABLE_TOKEN}',
      'Content-Type': 'application/json'
    }

    params = {
        "pageSize": 5
    }

    if offset:
        params["offset"] = offset

    response = requests.request("GET", url, headers=headers, params=params)

    return response

The next cell utilizes `get_golf_scores_by_page` by checking for the presence of `offset` in the response indicating there are more records in Airtable meeting your criteria to be retrieved. The code iteratively submits requests and stores the results in `all_records` until all the records are retrieved from Airtable.

In [64]:
# Paginate results.
all_records = []
results = get_golf_scores_by_page()
all_records.extend(results.json()["records"])

while "offset" in results.json():
    results = get_golf_scores_by_page(results.json()["offset"])
    all_records.extend(results.json()["records"])

You can also apply filters to your records in Airtable by defining the filters and behavior within the parameters of your request. The `get_scores_for_hole` function allows you to return records for only the hole you specify. In addition, only the fields for `Hole` and `Score` will be returned as a part of the output.

In [65]:
def get_scores_for_hole(hole):
    """Get scores for a specific hole."""
    url = f"{AIRTABLE_URL}/golf-scores"
    headers = {
      'Authorization': f'Bearer {AIRTABLE_TOKEN}',
      'Content-Type': 'application/json'
    }

    params = {
        "fields": ["Hole", "Score"],
        "filterByFormula": f"Hole={hole}"
    }

    response = requests.request("GET", url, headers=headers, params=params)

    return response

Let's test this function by getting the score for the third hole.

In [66]:
# Filter rows.
hole3 = get_scores_for_hole(3)

hole3.json()

{'records': [{'id': 'recH9472sCeg94tp2',
   'fields': {'Hole': 3, 'Score': 6},
   'createdTime': '2021-10-20T23:02:47.000Z'},
  {'id': 'recN005xRqgCaEAyb',
   'fields': {'Hole': 3, 'Score': 6},
   'createdTime': '2021-10-20T22:41:28.000Z'}]}

Unlike the previous examples where we returned data, this time you only get the records for hole 3 that contain just the `Hole` and `Score`.

For conveinance in the later requests I'm creating a variable `hole_3_id` that is equal to the `id` of the output from the last response. This ID is going to be needed for the next few actions. When you see this variable this is what it is referring to.

In [67]:
# Setting a variable for use later.
hole_3_id = hole3.json()["records"][0]["id"]

## Updating Records in Airtable

Once you have existing records in Airtable you may need to update a value of that record. You can do this through either a `PATCH` or `PUT` request.

Let's look at the `PATCH` request first. A `PATCH` request will update the values of a record you specify in your request and leave all other fields the same. To make a `PATCH` request for a record in Airtable you need to know the `id` of the record you want to update and then you can provide updated field values for the record.

The `update_record_fields` function submits a `PATCH` request.

In [68]:
def update_record_fields(updated_records):
    """Updates the records."""
    url = f"{AIRTABLE_URL}/golf-scores"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_TOKEN}',
        'Content-Type': 'application/json'
    }

    response = requests.request("PATCH", url, headers=headers, data=json.dumps(updated_records))

    return response

Let's update the score of the third hole to be 5 instead of 6. The `update_records` variable shows the syntax of the `PATCH` payload. Make sure you update the `id` field with the value with the proper record ID you returned when you retrieved them from Airtable.

In [69]:
# Update a record (PATCH).
update_records = {
    "records": [
        {
            "id": hole_3_id,
            "fields": {
                "Score": 5
            }
        }
    ]
}

response = update_record_fields(update_records)

Checking the score for hole 3 again you can see it has been updated to 5 and that all of the other fields have the same value.

In [70]:
# Check that records are updated.
hole3 = get_scores_for_hole(3)

hole3.json()

{'records': [{'id': 'recH9472sCeg94tp2',
   'fields': {'Hole': 3, 'Score': 5},
   'createdTime': '2021-10-20T23:02:47.000Z'},
  {'id': 'recN005xRqgCaEAyb',
   'fields': {'Hole': 3, 'Score': 6},
   'createdTime': '2021-10-20T22:41:28.000Z'}]}

Another way of updating records is by using a `PUT` request. Updating records via a `PUT` request is similar to using `PATCH` with one big difference in the outcome. Where a `PATCH` request updates the values you specify in your request and leaves the others the same a `PUT` request will update the values you specify in the payload and clear the values of all other fields.

Let's look at an example with a new function `replace_record_fields` that will submit the `PUT` request.

In [71]:
def replace_record_fields(updated_records):
    """Updates the records."""
    url = f"{AIRTABLE_URL}/golf-scores"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_TOKEN}',
        'Content-Type': 'application/json'
    }

    response = requests.request("PUT", url, headers=headers, data=json.dumps(updated_records))

    return response

The syntax of the payload is the same as the `PATCH` request, but this time let's change the score to 4.

In [72]:
# Update a record (PUT).
update_records = {
    "records": [
        {
            "id": hole_3_id,
            "fields": {
                "Score": 4
            }
        }
    ]
}

response = replace_record_fields(update_records)

Now, let's look at the output in Airtable. You should see one record with a `Score` of 4, but all the other fields are empty. That's the result of the `PUT` request. Since only a value for `Score` was specified in your payload that field is updated and all the others are cleared.

## Deleting Records

The last action we'll look at is deleting records from Airtable. To delete a record from Airtable you need to know the record IDs you want to delete. Those will be passed as query parameters with the `DELETE` request and Airtable will remove those records from your table.

The `delete_records` function below shows an example function.

In [73]:
def delete_records(records):
    """Deletes records from the table."""
    url = f"{AIRTABLE_URL}/golf-scores"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_TOKEN}',
        'Content-Type': 'application/json'
    }

    params = {
        "records[]": records
    }

    response = requests.request("DELETE", url, headers=headers, params=params)

    return response

In this last step let's delete the record for hole 3. Use the same `id` from the last step.

In [74]:
# Delete a record.
delete_ids = [hole_3_id]
response = delete_records(delete_ids)

If you look at your Airtable you should see that the record that only contained the score value has been deleted.